In [1]:
import pandas as pd

# set up preamble to have all required packages and definitions
preamble = r"""\documentclass[10pt, twocolumn, letterpaper]{article}
\usepackage[pass, margin=1in]{geometry} %one inch margins
\usepackage{ibpsa} % defines look
\usepackage{pslatex}
\usepackage{achicago}
\usepackage{amsmath, amssymb}
\usepackage{graphicx}
\usepackage{fancyhdr}
\usepackage{ifthen}
\usepackage[dvipsnames]{xcolor}
\usepackage{makecell}
\usepackage{longtable}

\pagestyle{empty}
\setlength{\voffset}{-1.25in}
\setlength{\headheight}{1in}
\definecolor{ibpsared}{RGB}{224, 2, 1}
\definecolor{headergray}{gray}{0.4}

\begin{document}"""

# BPACS headers have ASHRAE and IBPSA logos (logo_new.jpg), and use gray font for conference info
bpacsHeader = r"""\fancypagestyle{empty}{%
  \fancyhf{}%
  \vspace{0.5in}%
  \fancyhead[L]{\includegraphics [height=0.7in] {images/logo_new.jpg}}%
  \fancyhead[R]{\textcolor{headergray}{CONF_WITH_LINEBREAKS \\ LOCATION \\ DATES}}
}"""

# SimBuild headers only have IBPSA logo (ibpsa.jpg), and use gray font for conference info
simbuildHeader = r"""\fancypagestyle{empty}{%
  \fancyhf{}%
  \vspace{0.5in}%
  \fancyhead[L]{\includegraphics [height=0.7in] {images/ibpsa.jpg}}%
  \fancyhead[R]{CONF_WITH_LINEBREAKS \\ LOCATION \\ DATES}
}"""

# make an empty page titled "Table of Contents"
intro = r"""\section*{Table of Contents}
\thispagestyle{empty}"""

conferenceInfo = {
    '2004': {   'conf': r'Proceedings of SimBuild Conference 2004\\1st conference of IBPSA-USA',
                'loc': 'Boulder, Colorado',
                'date': 'August 4--6, 2004',
                'ashrae': False},
    '2006': {   'conf': r'Proceedings of SimBuild Conference 2006\\2nd Conference of IBPSA-USA',
                'loc': 'Cambridge, Massachusetts',
                'date': 'August 2--4, 2006',
                'ashrae': False},
    '2008': {   'conf': r'Proceedings of SimBuild Conference 2008\\3rd Conference of IBPSA-USA',
                'loc': 'Berkeley, California ',
                'date': 'July 30--August 01, 2008',
                'ashrae': False},
    '2010': {   'conf': r'Proceedings of SimBuild Conference 2010\\4th Conference of IBPSA-USA',
                'loc': 'New York City, New York',
                'date': 'August 11--13, 2010',
                'ashrae': False},
    '2012': {   'conf': r'Proceedings of SimBuild Conference 2012\\5th Conference of IBPSA USA',
                'loc': 'Madison, Wisconsin',
                'date': 'August 1--3, 2012',
                'ashrae': False},
    '2014': {   'conf': r'2014 ASHRAE/IBPSA-USA Building Simulation Conference',
                'loc': 'Atlanta, Georgia',
                'date': 'September 10--12, 2014',
                'ashrae': True},
    '2016': {   'conf': r'ASHRAE & IBPSA-USA SimBuild 2016\\Building Performance Modeling Conference',
                'loc': 'Salt Lake City, Utah',
                'date': 'August 10--12, 2016',
                'ashrae': True},
    '2018': {   'conf': r'2018 Building Performance Analysis Conference and SimBuild \\co-organized by ASHRAE and IBPSA-USA ',
                'loc': 'Chicago, Illinois',
                'date': 'September 26--28, 2018',
                'ashrae': True},
    '2020': {   'conf': r'2020 Building Performance Analysis Conference and SimBuild \\co-organized by ASHRAE and IBPSA-USA ',
                'loc': 'virtual',
                'date': 'September 29--October 1, 2020',
                'ashrae': True},
    '2022': {   'conf': r'2022 Building Performance Analysis Conference and SimBuild \\co-organized by ASHRAE and IBPSA-USA ',
                'loc': 'Chicago, Illinois',
                'date': 'September 14--16, 2022',
                'ashrae': True},
    '2024': {   'conf': r'Proceedings of SimBuild Conference 2024\\Eleventh National Conference of IBPSA-USA ',
                'loc': 'Denver, Colorado',
                'date': 'May 21--23, 2024',
                'ashrae': False}
}

def makeHeader(year):
    """Make a header that contains conference name, location, and dates, as well as relevant logos."""
    info = conferenceInfo[year]
    if info['ashrae'] == True:
        header = bpacsHeader.replace('CONF_WITH_LINEBREAKS', info['conf'])
        header = header.replace('LOCATION', info['loc'])
        header = header.replace('DATES', info['date'])
        return header
    else:
        header = simbuildHeader.replace('CONF_WITH_LINEBREAKS', info['conf'])
        header = header.replace('LOCATION', info['loc'])
        header = header.replace('DATES', info['date'])
        return header

def confIDToYear(confID):
    """Extract year from conference ID"""
    try:
        return confID.split('simbuild')[1]
    except:
        return ""
    
def formatTitle(title):
    """Update formatting for title to change color and escape characters that break LaTeX."""
    title = title.replace('&', '\&')
    title = title.replace('%', '\%')
    # if the title is too long, break it into multiple lines
    if len(title) > 86:
        parts = title.split(' ')
        lines = [r"\textcolor{ibpsared}{"]
        # add words to line until line is full, then start a new line
        for part in parts:
            if lines[-1] == r"\textcolor{ibpsared}{":
                lines[-1] = lines[-1] + part
            elif len(lines[-1] + part) + 1 < 108:
                lines[-1] = lines[-1] + ' ' + part
            else:
                lines[-1] = lines[-1] + r'}\\\textcolor{ibpsared}{' 
                lines.append(part)
        title = ''.join(lines)

        return title + r"}"
    else:
        return r"\textcolor{ibpsared}{" + title + r"}"
    
def formatAuthors(authors):
    """Change formatting for authors for the table of contents (first name last name, no affiliations, and line breaks to fit format)."""
    # commas in original file have names in reverse order
    authors = authors.replace(',', '')
    # remove affiliation notes
    authors = authors.replace(' (1)', '')
    authors = authors.replace(' (2)', '')
    authors = authors.replace(' (3)', '')
    authors = authors.replace(' (4)', '')
    authors = authors.replace(' (1,2)', '')
    authors = authors.replace(' (1,3)', '')
    authors = authors.replace(' (2,3)', '')
    authors = authors.replace(' (3,4)', '')
    # if the line length is too long, split into multiple lines
    if len(authors) > 86:
        parts = authors.split('; ')
        lines = ['']
        # add authors to line until line is full, then start a new line
        for part in parts:
            if len(lines[-1]) == 0:
                lines[-1] = part
            elif len(lines[-1] + part) + 1 < 86:
                lines[-1] = lines[-1] + ' ' + part
            else:
                lines[-1] = lines[-1] + r'}\\\textit{' 
                lines.append(part)
        return ''.join(lines)
    else:
        return authors

def makeTable(df, year):
    """Assemble table of contents for a specified conference year from individual parts"""
    # make header and start table
    header = makeHeader(year)
    tableStrs = [preamble, '', intro, '', header, '', r"\begin{longtable}{p{6in}r}"]
    # for each paper, add title, authors and page number of first page
    for index, row in df.iterrows():
        page = row['Page Start']
        authors = row['Authors']
        authors = formatAuthors(authors)
        title = row['Title']
        title = formatTitle(title)
        linebreak = r"\\"
        tableStrs.append(r"\begin{tabular}[c]{@{}l@{}}" + title + r"\\\textit{" + authors + r"}\end{tabular} & " + str(page) + linebreak)
    # wrap up table and document
    tableStrs[-1] = tableStrs[-1][:-2]
    tableStrs.append(r"\end{longtable}")
    tableStrs.append('')
    tableStrs.append(r"\end{document}")
    # concatente all content into a single string and write out to LaTeX file
    fullTable = '\n'.join(tableStrs)
    yearTex = open('simbuild/{}.tex'.format(year), 'w')
    yearTex.writelines(fullTable)
    yearTex.close()

df = pd.read_csv("SimBuild 2004-2024 - Paper Metadata.csv")

df["Year"] = df["Conference Id"].apply(confIDToYear)

for year in df["Year"]:
    yearDF = df[df["Year"] == year]
    makeTable(yearDF, year)